<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
#грузим библиотеки

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from tqdm import notebook
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')


from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve, f1_score

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
pd.options.display.max_colwidth = 350 # чтобы лучше текст прочитать
CV = 3
RANDOM_STATE = 12345
TEST_SIZE = 0.2

In [3]:
import os

pth1 = '/datasets/toxic_comments.csv'
pth2 = 'C:Downloads//toxic_comments.csv'
if os.path.exists(pth1):
    df = pd.read_csv(pth1)
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
else:
    print('Something is wrong')

In [4]:
df.shape

(159571, 2)

In [5]:
df.head()

,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences...",0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0


In [6]:
df.duplicated().sum()

0

**Посмотрим количество коментариев токсичных и нет. Так же взглянем на разметку , которую нам предоставили.**

In [7]:
for i in df[df.toxic == 0]['text'].head(5):
    print(i)

Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)
Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.
"
More
I can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want 

In [8]:
for i in df[df.toxic == 1]['text'].head(5):
    print(i)

COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
Hey... what is it..
@ | talk .
What is it... an exclusive group of some WP TALIBANS...who are good at destroying, self-appointed purist who GANG UP any one who asks them questions abt their ANTI-SOCIAL and DESTRUCTIVE (non)-contribution at WP?

Ask Sityush to clean up his behavior than issue me nonsensical warnings...
Bye! 

Don't look, come or think of comming back! Tosser.
You are gay or antisemmitian? 

Archangel WHite Tiger

Meow! Greetingshhh!

Uh, there are two ways, why you do erased my comment about WW2, that holocaust was brutally slaying of Jews and not gays/Gypsys/Slavs/anyone...

1 - If you are anti-semitian, than shave your head bald and go to the skinhead meetings!

2 - If you doubt words of the Bible, that homosexuality is a deadly sin, make a pentagram tatoo on your forehead go to the satanistic masses with your gay pals!

3 - First and last warning, you fucking gay - I won't appreciate if any more nazi shwain would write in 

In [9]:
sentense_example = df.iloc[1]['text']
# разбиваем на токены
tokens = word_tokenize(sentense_example, language='english')
# удаляем пунктуацию т.к. не модель будет хуже работать
tokens_without_punktuation = [i for i in tokens if i not in string.punctuation]
english_stop_words = stopwords.words('english')
# в этом пункте она уже без стоп слов и пунктуации
tokens_without_stop_words_and_punctuation = [i for i in tokens_without_punktuation if i not in english_stop_words]
# приводим к нижнему регистру и удаляем окончания
snowball = SnowballStemmer(language='english')
stemmed_tokens = [snowball.stem(i) for i in tokens_without_stop_words_and_punctuation]

In [10]:
print(f'Исходный текст: {sentense_example}')
print('--------------------')
print(f'Токены: {tokens}')
print('--------------------')
print(f'Токены без пунктуации: {tokens_without_punktuation}')
print('--------------------')
print(f'Токены без пунктуации и стоп слов: {tokens_without_stop_words_and_punctuation}')
print('--------------------')
print(f'Токены после стеминга: {stemmed_tokens}')
print('--------------------')

Исходный текст: D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)
--------------------
Токены: ["D'aww", '!', 'He', 'matches', 'this', 'background', 'colour', 'I', "'m", 'seemingly', 'stuck', 'with', '.', 'Thanks', '.', '(', 'talk', ')', '21:51', ',', 'January', '11', ',', '2016', '(', 'UTC', ')']
--------------------
Токены без пунктуации: ["D'aww", 'He', 'matches', 'this', 'background', 'colour', 'I', "'m", 'seemingly', 'stuck', 'with', 'Thanks', 'talk', '21:51', 'January', '11', '2016', 'UTC']
--------------------
Токены без пунктуации и стоп слов: ["D'aww", 'He', 'matches', 'background', 'colour', 'I', "'m", 'seemingly', 'stuck', 'Thanks', 'talk', '21:51', 'January', '11', '2016', 'UTC']
--------------------
Токены после стеминга: ["d'aww", 'he', 'match', 'background', 'colour', 'i', "'m", 'seem', 'stuck', 'thank', 'talk', '21:51', 'januari', '11', '2016', 'utc']
--------------------


## Обучение

In [11]:
snowball = SnowballStemmer(language='english')
english_stop_words = stopwords.words('english')
# функция для общего массива
def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language='english')
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in english_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [12]:
tokenize_sentence(sentense_example) # проверим работу функции на 1 примере

["d'aww",
 'he',
 'match',
 'background',
 'colour',
 'i',
 "'m",
 'seem',
 'stuck',
 'thank',
 'talk',
 '21:51',
 'januari',
 '11',
 '2016',
 'utc']

In [13]:
train_df, test_df = train_test_split(df, test_size=TEST_SIZE)
display(test_df.shape)
display(train_df.shape)

(31915, 2)

(127656, 2)

In [14]:
display(train_df.toxic.value_counts())
display(test_df.toxic.value_counts())

0    114593
1     13063
Name: toxic, dtype: int64

0    28753
1     3162
Name: toxic, dtype: int64

In [15]:
vec = TfidfVectorizer(tokenizer= lambda x: tokenize_sentence(x, remove_stop_words=True))

In [16]:
features = vec.fit_transform(train_df.text)

In [ ]:
model = LogisticRegression(random_state=12345).fit(features, train_df.toxic)

Посмотрим коммент,модель нам показала негатив проверим его и убедимся,что она верно предсказала

In [ ]:
model.predict(features[0])

In [ ]:
train_df.text.iloc[0] # Умри \n\nЯ НЕНАВИЖУ ТЕБЯ, ЧУДО, ТЫ НЕ ЗАСЛУЖИВАЕШЬ ЗДЕСЬ МЕСТА'

In [ ]:
model_pipeline = Pipeline([
    ('vec', TfidfVectorizer(tokenizer= lambda x: tokenize_sentence(x, remove_stop_words=True))),
    ('model', LogisticRegression(random_state=12345))
])

In [ ]:
model_pipeline.fit(train_df.text, train_df.toxic)

После обучения проверим модель сами напишем рандомный коментарий и негативный

In [ ]:
model_pipeline.predict(['Hi, is it okay bro. I gonna talk with Alfk'])

In [ ]:
model_pipeline.predict(['Lets go to fucking outside'])

знаем, что нам нужна f1 мера, но без подкрутки точности и полноты мы не добьёмся нужного результат.
Следовательно будем смотреть наши результаты Precision and Recall тоже.

In [ ]:
print(f'предсказания точности на тестовой выборке равен: {precision_score(y_true=test_df.toxic,y_pred=model_pipeline.predict(test_df.text))}')
print(f'предсказания полноты на тестовой выборке равен: {recall_score(y_true=test_df.toxic,y_pred=model_pipeline.predict(test_df.text))}')
print(f'предсказания f1 меры на тестовой выборке равен: {f1_score(y_true=test_df.toxic,y_pred=model_pipeline.predict(test_df.text))}')

In [ ]:
prec, rec, thresholds = precision_recall_curve(y_true=test_df.toxic, probas_pred=model_pipeline.predict_proba(test_df.text)[:,1])

In [ ]:
plot_precision_recall_curve(estimator=model_pipeline, X=test_df.text, y=test_df.toxic)
plt.show()

можно попробовать подкурить параметры , если не выйдет то будем передвигаться к подбору гиперпараметров.

In [ ]:
np.where(prec>0.95)

In [ ]:
thresholds[27777]

In [ ]:
precision_score(y_true=test_df.toxic, y_pred=model_pipeline.predict_proba(test_df.text)[:,1] > thresholds[27111])

In [ ]:
f1_score(y_true=test_df.toxic, y_pred=model_pipeline.predict_proba(test_df.text)[:,1] > thresholds[25397])

подберем гиперпараметры для нашей логистической регрессии.

In [ ]:
grid_pipeline = Pipeline([
    ('vec', TfidfVectorizer(tokenizer= lambda x: tokenize_sentence(x, remove_stop_words=True))),
    ('model', GridSearchCV(
    LogisticRegression(random_state=12345),
    param_grid={'C': [0.1, 1, 10.]},
    cv=3,
    verbose=4
    )
     )
])

In [ ]:
grid_pipeline.fit(train_df.text, train_df.toxic)

In [ ]:
model_pipeline_up = Pipeline([
    ('vec', TfidfVectorizer(tokenizer= lambda x: tokenize_sentence(x, remove_stop_words=True))),
    ('model', LogisticRegression(random_state=12345,C=10.))
])

In [ ]:
model_pipeline_up.fit(train_df.text, train_df.toxic)

In [ ]:
prec_up, rec_up, thresholds_up = precision_recall_curve(y_true=test_df.toxic, probas_pred=model_pipeline_up.predict_proba(test_df.text)[:,1])

In [ ]:
plot_precision_recall_curve(estimator=model_pipeline_up, X=test_df.text, y=test_df.toxic)
plt.show()

## Выводы

In [37]:
print('Итоговый результаты Логистической регрессии на тестовой выборке')
print('                                                                 ')
print(f'предсказания точности на тестовой выборке равен: {precision_score(y_true=test_df.toxic,y_pred=model_pipeline_up.predict(test_df.text))}')
print(f'предсказания полноты на тестовой выборке равен: {recall_score(y_true=test_df.toxic,y_pred=model_pipeline_up.predict(test_df.text))}')
print(f'предсказания f1 меры на тестовой выборке равен: {f1_score(y_true=test_df.toxic,y_pred=model_pipeline_up.predict(test_df.text))}')

Итоговый результаты Логистической регрессии на тестовой выборке
                                                                 
предсказания точности на тестовой выборке равен: 0.8802678219771564
предсказания полноты на тестовой выборке равен: 0.6982193064667291
предсказания f1 меры на тестовой выборке равен: 0.7787456445993031


1. Посмотрели на наши данные определи, что качество разметки данных отличное для выполнения поставленной задачи.

2. Подготовили нашу модель для обучения.

3. Провели Лемматизация данных.

4. Проверили точность и полноту модели на тестовой выборке.

5. Нашли наилучшие параметры на `LogisticRegression`.

*Мы добились результата на логистической регрессии `f1 меры 0.77` и выполнили задание заказчика !*